In [1]:
import numpy as np
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.clustering import KMeans

#generate random data RDD we need this package
from pyspark.mllib.random import RandomRDDs

In [2]:
import pyspark
sc = pyspark.SparkContext('spark://spark-master:7077')

In [3]:
c1_v = RandomRDDs.normalVectorRDD(sc,20,2,numPartitions=2,seed=1).map(lambda v:np.add([1,5],v))
c2_v = RandomRDDs.normalVectorRDD(sc,16,2,numPartitions=2,seed=2).map(lambda v:np.add([5,1],v))
c3_v = RandomRDDs.normalVectorRDD(sc,12,2,numPartitions=2,seed=3).map(lambda v:np.add([4,6],v))

#concatenate 2 RDDs with  .union(other) function
c12 = c1_v.union(c2_v)
my_data = c12.union(c3_v)   #this now has all points, as RDD


my_kmmodel = KMeans.train(my_data,k=1,
               maxIterations=20,runs=1,
               initializationMode='k-means||',seed=10)

In [4]:
def getsse(point):
    this_center = my_kmmodel.centers[my_kmmodel.predict(point)]
           #for this point get it's clustercenter
    return (sum([x**2 for x in (point - this_center)])) 


my_sse = my_data.map(getsse).collect()  # collect list of sse of each pt to its center
print(np.array(my_sse).mean())

9.57719005617


In [5]:
my_kmmodel = KMeans.train(my_data,k=4,
               maxIterations=20,runs=1,
               initializationMode='k-means||',seed=10)

my_sse = my_data.map(getsse).collect()  # collect list of sse of each pt to its center
print(np.array(my_sse).mean())

1.73017164023


In [6]:
my_data.stats()

(count: 48, mean: [ 3.12801059  3.92940785], stdev: [ 2.11814298  2.25624918], max: [ 6.36840832  8.04523732], min: [-1.33872715 -0.32867964])

In [7]:
my_kmmodel = KMeans.train(my_data,k=3,
               maxIterations=20,runs=1,
               initializationMode='k-means||',seed=10)

my_sse = my_data.map(getsse).collect()  # collect list of sse of each pt to its center
print(np.array(my_sse).mean())

2.10255158159
